In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse(r'C:\Users\cid2r\Downloads\db\discogs_20220201_masters.xml')
#tree = ET.parse('rel.xml')
root = tree.getroot()

In [3]:
for child in root:
    print(child.tag, child.attrib)

master {'id': '113'}
master {'id': '114'}
master {'id': '115'}
master {'id': '116'}
master {'id': '117'}
master {'id': '118'}
master {'id': '119'}
master {'id': '120'}
master {'id': '121'}
master {'id': '122'}
master {'id': '123'}
master {'id': '124'}
master {'id': '125'}
master {'id': '126'}
master {'id': '127'}
master {'id': '128'}
master {'id': '129'}
master {'id': '130'}
master {'id': '131'}
master {'id': '132'}
master {'id': '133'}
master {'id': '135'}
master {'id': '136'}
master {'id': '137'}
master {'id': '138'}
master {'id': '139'}
master {'id': '140'}
master {'id': '141'}
master {'id': '142'}
master {'id': '143'}
master {'id': '144'}
master {'id': '145'}
master {'id': '146'}
master {'id': '147'}
master {'id': '148'}
master {'id': '149'}
master {'id': '150'}
master {'id': '151'}
master {'id': '152'}
master {'id': '154'}
master {'id': '155'}
master {'id': '156'}
master {'id': '157'}
master {'id': '158'}
master {'id': '160'}
master {'id': '162'}
master {'id': '163'}
master {'id':

KeyboardInterrupt: 

In [4]:

import pandas as pd

import sys
if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


In [ ]:
pip install python3-discogs-client

In [4]:
rows = []

for node in root:
    n_title = node.attrib["id"]
    #n_genre = node.find("genre").text
    rows.append({"title":n_title})
    

KeyboardInterrupt: 

In [5]:
import discogs_client

d = discogs_client.Client('ExampleApplication/0.1', user_token = 'WkcTWMWNFmwUFYHrtQIreHRnLmyFZTNnkflNHsgc')


In [10]:
d.master(121).stats

AttributeError: 'Master' object has no attribute 'stats'

In [8]:
master = []
masters = []


for release in root.iter('master'):
    id = release.attrib.get('id')
    if(int(id) > 84999):
        print(id)
    
        gens=[]
        for gen in release.iter('genre'):
           gens.append(gen.text)
    
        styls=[]
        for sty in release.iter('style'):
            styls.append(sty.text)
    
        master=[]
        imglnk = d.master(id).images
        if(imglnk != None):
            if('uri150' in imglnk[0]):
                uri = imglnk[0]['uri150']
    
                master = [id, gens, styls, uri]
    
                masters.append(master)

    
    if(int(id) > 99999):
        break

print("done! done!")

df = pd.DataFrame(masters, columns=['id', 'genre', 'styles', 'uri'])

from pandas import DataFrame

df.to_csv(r'C:\Users\cid2r\Documents\!VSCode\0220dbtest\data_85k_2_100k.csv')

85000
85001
85002
85003
85004
85005
85006
85007
85008
85009
85011
85012
85013
85014
85015
85017
85018
85019
85020
85021
85022
85023
85024
85025
85026
85027
85028
85029
85030
85031
85032
85033
85034
85035
85036
85037
85038
85039
85040
85041
85042
85043
85044
85045
85046
85047
85048
85049
85050
85051
85052
85053
85054
85055
85056
85057
85058
85059
85060
85061
85062
85063
85064
85065
85066
85067
85068
85069
85070
85071
85072
85073
85074
85075
85076
85077
85078
85079
85080
85081
85082
85083
85084
85085
85086
85087
85088
85089
85090
85091
85092
85093
85094
85095
85096
85097
85098
85099
85100
85101
85102
85103
85105
85106
85107
85108
85109
85110
85111
85112
85113
85114
85115
85116
85117
85118
85119
85120
85121
85122
85123
85124
85125
85126
85127
85128
85129
85130
85131
85132
85133
85134
85135
85136
85137
85138
85139
85140
85141
85142
85143
85144
85145
85146
85148
85149
85150
85151
85152
85153
85154
85155
85156
85157
85158
85159
85160
85161
85162
85163
85164
85165
85166
85167
85168
85169
8517

In [5]:
import os

db_list = os.listdir(r"C:\Users\cid2r\Documents\!VSCode\0220dbtest\data")

In [15]:
cd data

c:\Users\cid2r\Documents\!VSCode\0220dbtest\data


In [65]:
import pandas as pd



df_0_2_100k = pd.DataFrame()

for file in db_list:
    df = pd.read_csv(file, dtype={'Code':str}, parse_dates=['id'])
    df_0_2_100k = pd.concat([df_0_2_100k, df])


In [68]:
df_0_2_100k

,Unnamed: 0,id,genre,styles,uri
0,0,113,['Electronic'],"['Techno', 'Tech House']",https://i.discogs.com/KHlEsv1u4r9X7QGdgI4qSm1u...
1,1,114,['Electronic'],"['Deep House', 'Tech House', 'Techno']",https://i.discogs.com/d1oKVlvPx5poaXMKEZkEkIdo...
2,2,115,['Electronic'],"['IDM', 'Electro']",https://i.discogs.com/vGW-RvJW--kn8dLWCBVAJuxN...
3,3,116,['Electronic'],['Techno'],https://i.discogs.com/h8PHh0IMmyIuIfcPGksAAMqi...
4,4,117,['Electronic'],['Progressive Trance'],https://i.discogs.com/vu5I9wU6lJafJAHSGFuYx3mT...
...,...,...,...,...,...
14413,14413,99996,['Reggae'],"['Reggae', 'Roots Reggae']",https://i.discogs.com/V-xPTlUYWiII79-smHlfA6z6...
14414,14414,99997,['Reggae'],['Roots Reggae'],https://i.discogs.com/FYRAhSe2xL2aD5tB13Cguui6...
14415,14415,99998,['Reggae'],['Roots Reggae'],https://i.discogs.com/1h6e_QBhvpmLk7EOJSGGoKKz...
14416,14416,99999,['Electronic'],['Progressive House'],https://i.discogs.com/1pa6F4S0Eq_d6MkuNknXJPTh...


In [73]:
df_test = df_0_2_100k.copy()

In [75]:
from ast import literal_eval


for idx, row in df_test.iterrows():
    genres = literal_eval(row['genre'])
    print(row)
    for genre in genres:
        print(genre)
    print("---")
            

Unnamed: 0                                                    0
id                                                          113
genre                                            ['Electronic']
styles                                 ['Techno', 'Tech House']
uri           https://i.discogs.com/KHlEsv1u4r9X7QGdgI4qSm1u...
Name: 0, dtype: object
Electronic
---
Unnamed: 0                                                    1
id                                                          114
genre                                            ['Electronic']
styles                   ['Deep House', 'Tech House', 'Techno']
uri           https://i.discogs.com/d1oKVlvPx5poaXMKEZkEkIdo...
Name: 1, dtype: object
Electronic
---
Unnamed: 0                                                    2
id                                                          115
genre                                            ['Electronic']
styles                                       ['IDM', 'Electro']
uri           https://i.disc

KeyboardInterrupt: 

In [61]:
if('id' in df_test.columns):
    print("yes")

In [74]:

print(df_test)

       Unnamed: 0      id           genre  \
0               0     113  ['Electronic']   
1               1     114  ['Electronic']   
2               2     115  ['Electronic']   
3               3     116  ['Electronic']   
4               4     117  ['Electronic']   
...           ...     ...             ...   
14413       14413   99996      ['Reggae']   
14414       14414   99997      ['Reggae']   
14415       14415   99998      ['Reggae']   
14416       14416   99999  ['Electronic']   
14417       14417  100000      ['Reggae']   

                                       styles  \
0                    ['Techno', 'Tech House']   
1      ['Deep House', 'Tech House', 'Techno']   
2                          ['IDM', 'Electro']   
3                                  ['Techno']   
4                      ['Progressive Trance']   
...                                       ...   
14413              ['Reggae', 'Roots Reggae']   
14414                        ['Roots Reggae']   
14415             

In [20]:
from pandas import DataFrame

df.to_csv(r'C:\Users\cid2r\Documents\!VSCode\0220dbtest\df.csv')

In [41]:
DISCOGS_KEY = 'OKxXReJmUJvMJRRsaOqr'
DISCOGS_SECRET = 'RormADbxpiujDCXDiBDwFGUFxKrVjkTK'

In [ ]:
from bs4 import BeautifulSoup
import re
import urllib.request
from urllib.request import urlopen
import pandas as pd

url = open(r"C:\Users\cid2r\Documents\!VSCode\0220dbtest\rel.xml")

soup = BeautifulSoup(url, "html.parser")


data = {'id' : [], 'genre' : []}


for release in soup.select('release'):
    data['id'].append(release['id'])
    for gen in release.select('genre'):
        data['genre'].append(gen)

asdf = pd.DataFrame(data)

print(data)

#print(asdf)


In [19]:


for child in root:
    if(child.find("master_id") != None):
        print("=" * 10)
        print(child.attrib["id"])
        for gens in child.iter("genres"):
            for gen in gens.findall("genre"):
                print(gen.text)
        for mr in child.iter("master_id"):
            print(mr.text)
        ##child.write("output.xml", encoding="utf-8", xml_declaration=True)
        

1
jazz
soul
100
3
rock
300
